In [8]:
!pip install kagglehub pandas scikit-learn fairlearn aif360 --quiet

In [1]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset
df = pd.read_csv(r"C:\Users\Arhamsoft\Desktop\Talha Talib Thesis\german_credit_data.csv")

# Drop missing values
df.dropna(inplace=True)

# Show current target values
print("Unique values in 'kredit':", df['kredit'].unique())

# Target and features
y = df['kredit']  # Already binary: 0 = Good, 1 = Bad
X = df.drop(columns=['kredit'])

# Encode categorical features
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include='object').columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col])

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Output results
print("\nRandom Forest Performance Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Unique values in 'kredit': [1 0]

Random Forest Performance Metrics:
Accuracy : 0.7733
Precision: 0.7886
Recall   : 0.9238
F1 Score : 0.8509


In [3]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Derive protected attribute: 0 = Female, 1 = Male
df['sex_binary'] = df['famges'].apply(lambda x: 0 if x == 4 else 1)

# 2. Combine features, labels, and protected attribute
full_data = pd.concat([X_encoded, y.rename('target'), df['sex_binary']], axis=1)

# 3. Create AIF360 BinaryLabelDataset
bld = BinaryLabelDataset(
    df=full_data,
    label_names=['target'],
    protected_attribute_names=['sex_binary'],
    favorable_label=1,  # Good credit
    unfavorable_label=0  # Bad credit
)

# 4. Train/Test split
train_bld, test_bld = bld.split([0.7], shuffle=True)

# 5. Apply Reweighing
rw = Reweighing(
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
train_rw_bld = rw.fit_transform(train_bld)

# 6. Extract training/testing inputs
X_train_rw = train_rw_bld.features
y_train_rw = train_rw_bld.labels.ravel()
w_train_rw = train_rw_bld.instance_weights

X_test_rw = test_bld.features
y_test_rw = test_bld.labels.ravel()

# 7. Train Random Forest on reweighed data
rf_rw = RandomForestClassifier(n_estimators=100, random_state=42)
rf_rw.fit(X_train_rw, y_train_rw, sample_weight=w_train_rw)

# 8. Predictions
y_pred_rw = rf_rw.predict(X_test_rw)

# 9. Performance Metrics
acc = accuracy_score(y_test_rw, y_pred_rw)
prec = precision_score(y_test_rw, y_pred_rw)
rec = recall_score(y_test_rw, y_pred_rw)
f1 = f1_score(y_test_rw, y_pred_rw)

print("Reweighed Random Forest Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# 10. Fairness Metrics
pred_bld_rw = test_bld.copy()
pred_bld_rw.labels = y_pred_rw.reshape(-1, 1)

metric_rw = ClassificationMetric(
    test_bld,
    pred_bld_rw,
    unprivileged_groups=[{'sex_binary': 0}],
    privileged_groups=[{'sex_binary': 1}]
)

spd = metric_rw.statistical_parity_difference()
eod = metric_rw.equal_opportunity_difference()
dir_ = metric_rw.disparate_impact()

print("\nFairness Metrics (Reweighing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {dir_:.4f}")


Reweighed Random Forest Performance:
Accuracy : 0.7733
Precision: 0.8058
Recall   : 0.9028
F1 Score : 0.8515

Fairness Metrics (Reweighing):
Statistical Parity Difference : 0.0788
Equal Opportunity Difference : -0.0283
Disparate Impact Ratio       : 1.0990


In [5]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Rebuild BinaryLabelDataset using existing data
X_encoded['sex_binary'] = df['sex_binary'].values
full_data_rf = pd.concat([X_encoded, y.rename("target")], axis=1)

bld_rf = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=full_data_rf,
    label_names=["target"],
    protected_attribute_names=["sex_binary"]
)

# Step 2: Split into train/test
train_bld_rf, test_bld_rf = bld_rf.split([0.7], shuffle=True)

# Step 3: Train random forest with reweighing weights
rf_eq = RandomForestClassifier(n_estimators=100, random_state=42)
rf_eq.fit(train_bld_rf.features, train_bld_rf.labels.ravel(), sample_weight=train_rw_bld.instance_weights)

# Step 4: Get predicted labels
rf_probs = rf_eq.predict_proba(test_bld_rf.features)[:, 1]
rf_preds = (rf_probs >= 0.5).astype(int)

# Step 5: Wrap predictions into BinaryLabelDataset
rf_pred_bld = test_bld_rf.copy()
rf_pred_bld.labels = rf_preds.reshape(-1, 1)

# Step 6: Apply Equalized Odds Postprocessing
eq_odds_rf = EqOddsPostprocessing(
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)
eq_odds_rf = eq_odds_rf.fit(test_bld_rf, rf_pred_bld)
rf_eq_pred = eq_odds_rf.predict(rf_pred_bld)

# Step 7: Extract predictions
y_pred_eq_rf = rf_eq_pred.labels.ravel()
y_true_eq_rf = test_bld_rf.labels.ravel()

# Step 8: Evaluation
acc = accuracy_score(y_true_eq_rf, y_pred_eq_rf)
prec = precision_score(y_true_eq_rf, y_pred_eq_rf)
rec = recall_score(y_true_eq_rf, y_pred_eq_rf)
f1 = f1_score(y_true_eq_rf, y_pred_eq_rf)

print("Equalized Odds Postprocessing (Random Forest) Performance:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Step 9: Fairness metrics
metric_rf_eq = ClassificationMetric(
    test_bld_rf,
    rf_eq_pred,
    privileged_groups=[{'sex_binary': 1}],
    unprivileged_groups=[{'sex_binary': 0}]
)

spd = metric_rf_eq.statistical_parity_difference()
eod = metric_rf_eq.equal_opportunity_difference()
dir_ratio = metric_rf_eq.disparate_impact()

print("\nFairness Metrics (Equalized Odds Postprocessing):")
print(f"Statistical Parity Difference : {spd:.4f}")
print(f"Equal Opportunity Difference : {eod:.4f}")
print(f"Disparate Impact Ratio       : {dir_ratio:.4f}")


Equalized Odds Postprocessing (Random Forest) Performance:
Accuracy : 0.7667
Precision: 0.7739
Recall   : 0.9733
F1 Score : 0.8622

Fairness Metrics (Equalized Odds Postprocessing):
Statistical Parity Difference : 0.0163
Equal Opportunity Difference : 0.0286
Disparate Impact Ratio       : 1.0173
